In [ ]:
# I want to predict the individual teams total points and add them together so build dataframes with teams offensive stats and opponents defensive stats

# Retrieve data, clean data and store in folder to train model on

In [ ]:
# Imports

import nflreadpy as nfl
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

## Retrieve data

In [ ]:
schedule = nfl.load_schedules(seasons=[2024, 2025])
schedule = schedule.to_pandas()
schedule = schedule[schedule['game_type'] == 'REG']

team_stats = nfl.load_team_stats(seasons=[2024, 2025])
team_stats = team_stats.to_pandas()
team_stats = team_stats[team_stats['season_type'] == 'REG']

In [ ]:
team_stats

,season,week,team,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,passing_interceptions,...,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance
0,2024,1,ARI,REG,BUF,21,31,162,1,0,...,2,2,0,0,1.0,0,0,0,0,0
1,2024,1,ATL,REG,PIT,16,26,155,1,2,...,1,1,0,0,1.0,0,0,0,0,0
2,2024,1,BAL,REG,KC,26,41,273,1,0,...,2,2,0,0,1.0,0,0,0,0,0
3,2024,1,BUF,REG,ARI,18,23,232,2,0,...,4,4,0,0,1.0,0,0,0,0,0
4,2024,1,CAR,REG,NO,13,31,161,0,2,...,1,1,0,0,1.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,2025,11,SEA,REG,LA,29,44,279,0,4,...,1,1,0,0,1.0,0,1,1,0,61
892,2025,11,SF,REG,ARI,19,26,200,3,0,...,2,4,1,1,0.5,0,0,0,0,0
893,2025,11,TB,REG,BUF,16,28,173,1,1,...,2,2,0,0,1.0,0,0,0,0,0
894,2025,11,TEN,REG,HOU,24,37,194,1,0,...,1,1,0,0,1.0,0,0,0,0,0


In [ ]:
# columns from schedule: week, home_team, away_team, home_score, away_score, total_line
schedule_interests = schedule[['game_id', 'season', 'week', 'home_team', 'home_score', 'away_team', 'away_score', 'total_line']]

In [ ]:
schedule_away = schedule_interests
schedule_home = schedule_interests

In [ ]:
schedule_away = schedule_away.rename(columns={'home_team':'opponent', 'away_team':'team', 'away_score':'points'})
schedule_away = schedule_away[['game_id', 'season', 'week', 'team', 'opponent', 'points', 'total_line']]

schedule_home = schedule_home.rename(columns={'away_team':'opponent', 'home_team':'team', 'home_score':'points'})
schedule_home = schedule_home[['game_id', 'season', 'week', 'team', 'opponent', 'points', 'total_line']]

In [ ]:
schedule_total = pd.concat([schedule_away, schedule_home])

In [ ]:
schedule_total

,game_id,season,week,team,opponent,points,total_line
0,2024_01_BAL_KC,2024,1,BAL,KC,20.0,46.0
1,2024_01_GB_PHI,2024,1,GB,PHI,29.0,49.5
2,2024_01_PIT_ATL,2024,1,PIT,ATL,18.0,43.0
3,2024_01_ARI_BUF,2024,1,ARI,BUF,28.0,46.0
4,2024_01_TEN_CHI,2024,1,TEN,CHI,17.0,43.0
...,...,...,...,...,...,...,...
552,2025_18_DAL_NYG,2025,18,NYG,DAL,NaN,NaN
553,2025_18_WAS_PHI,2025,18,PHI,WAS,NaN,NaN
554,2025_18_BAL_PIT,2025,18,PIT,BAL,NaN,NaN
555,2025_18_SEA_SF,2025,18,SF,SEA,NaN,NaN


In [ ]:
# Filter out data I don't want

cols_to_drop = [
 'season_type',
 'sack_fumbles',
 'passing_air_yards',
 'passing_yards_after_catch',
 'passing_first_downs',
 'passing_2pt_conversions',
 'rushing_2pt_conversions',
 'receptions',
 'targets',
 'receiving_yards',
 'receiving_tds',
 'receiving_fumbles',
 'receiving_air_yards',
 'receiving_yards_after_catch',
 'receiving_first_downs',
 'receiving_2pt_conversions',
 'special_teams_tds',
 'misc_yards',
 'fumble_recovery_own',
 'fumble_recovery_yards_own',
 'fumble_recovery_opp',
 'fumble_recovery_yards_opp',
 'fumble_recovery_tds',
 'timeouts',
 'fg_made',
 'fg_missed',
 'fg_blocked',
 'fg_long',
 'fg_made_0_19',
 'fg_made_20_29',
 'fg_made_30_39',
 'fg_made_40_49',
 'fg_made_50_59',
 'fg_made_60_',
 'fg_missed_0_19',
 'fg_missed_20_29',
 'fg_missed_30_39',
 'fg_missed_40_49',
 'fg_missed_50_59',
 'fg_missed_60_',
 'fg_made_list',
 'fg_missed_list',
 'fg_blocked_list',
 'fg_made_distance',
 'fg_missed_distance',
 'fg_blocked_distance', 
 'pat_made',
 'pat_att',
 'pat_missed',
 'pat_blocked',
 'gwfg_made',
 'gwfg_att',
 'gwfg_missed',
 'gwfg_blocked',
 'gwfg_distance']

team_stats = team_stats.drop(columns=cols_to_drop)

In [ ]:
# Add points so I can average them for future weeks predictions
team_stats = team_stats.merge(schedule_total[['season', 'week', 'team', 'points']], on=['season', 'week', 'team'], how='left')
team_stats = team_stats.rename(columns={'points': 'points_scored'})
team_stats = team_stats.merge(schedule_total[['season', 'week', 'team', 'points']], left_on=['season', 'week', 'opponent_team'], right_on=['season', 'week','team'], how='left')
team_stats = team_stats.rename(columns={'points': 'points_against'})
team_stats = team_stats.rename(columns={'team_x': 'team'})
team_stats = team_stats.drop(columns=['team_y'])

,season,week,team,opponent_team,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,...,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_att,fg_pct,pat_pct,points_scored,points_against
0,2024,1,ARI,BUF,21,31,162,1,0,4,...,31,1,6,3,123,2,1.000000,1.0,28.0,34.0
1,2024,1,ATL,PIT,16,26,155,1,2,2,...,34,2,28,2,56,1,1.000000,1.0,10.0,18.0
2,2024,1,BAL,KC,26,41,273,1,0,1,...,64,1,3,0,0,3,0.666667,1.0,20.0,27.0
3,2024,1,BUF,ARI,18,23,232,2,0,2,...,65,1,7,1,53,2,1.000000,1.0,34.0,28.0
4,2024,1,CAR,NO,13,31,161,0,2,4,...,80,0,0,9,232,1,1.000000,1.0,10.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,2025,11,SEA,LA,29,44,279,0,4,0,...,47,2,20,0,0,5,0.800000,1.0,19.0,21.0
866,2025,11,SF,ARI,19,26,200,3,0,1,...,15,1,9,2,140,3,1.000000,0.5,41.0,22.0
867,2025,11,TB,BUF,16,28,173,1,1,1,...,30,0,0,6,161,2,1.000000,1.0,32.0,44.0
868,2025,11,TEN,HOU,24,37,194,1,0,3,...,62,2,1,2,49,2,1.000000,1.0,13.0,16.0


In [ ]:
# for each week group by team sum/avg all previous weeks , separate into 2024 and 2025 seasons
team_stats_24 = team_stats[team_stats['season'] == 2024]
team_stats_25 = team_stats[team_stats['season'] == 2025]
team_stats_24 = team_stats_24.sort_values(["team", "week"]).reset_index(drop=True)
team_stats_25 = team_stats_25.sort_values(["team", "week"]).reset_index(drop=True)

In [ ]:
# Cumulative averages up to a certain week

avg_cols = ['passing_epa',
 'passing_cpoe',
 'rushing_epa',
 'receiving_epa',
 'fg_pct',
 'pat_pct',
 'completions',
 'attempts',
 'passing_yards',
 'passing_tds',
 'passing_interceptions',
 'sacks_suffered',
 'sack_yards_lost',
 'sack_fumbles_lost',
 'carries',
 'rushing_yards',
 'rushing_tds',
 'rushing_fumbles_lost',
 'rushing_first_downs',
 'receiving_fumbles_lost',
 'def_tackles_solo',
 'def_tackles_with_assist',
 'def_tackle_assists',
 'def_tackles_for_loss',
 'def_tackles_for_loss_yards',
 'def_fumbles_forced',
 'def_sacks',
 'def_sack_yards',
 'def_qb_hits',
 'def_interceptions',
 'def_interception_yards',
 'def_pass_defended',
 'def_tds',
 'def_fumbles',
 'def_safeties',
 'penalties',
 'penalty_yards',
 'punt_returns',
 'punt_return_yards',
 'kickoff_returns',
 'kickoff_return_yards',
 'fg_att', 
 'points_scored',
 'points_against']


In [ ]:
team_stats_24['week'] = team_stats_24['week'].astype(int)

# Get all unique teams and full week range
teams = team_stats_24['team'].unique()
weeks = range(1, team_stats_24['week'].max() + 1)

# Create a MultiIndex of all possible (team, week) combinations
full_index = pd.MultiIndex.from_product([teams, weeks], names=['team', 'week'])

# Reindex the dataframe to include all rows
team_stats_24 = team_stats_24.set_index(['team', 'week']).reindex(full_index).reset_index()

In [ ]:
# Compute cumulative mean for previous games (excluding current game)
team_stats_24[[f"avg_{c}" for c in avg_cols]] = (
    team_stats_24.sort_values(by=['team', 'week'])
    .groupby("team")[avg_cols]
      .expanding()
      .mean()
      .shift(periods=1)
      .reset_index(level=0, drop=True)
)

In [ ]:
team_stats_25['week'] = team_stats_25['week'].astype(int)

# Get all unique teams and full week range
teams = team_stats_25['team'].unique()
weeks = range(1, team_stats_25['week'].max() + 1)

# Create a MultiIndex of all possible (team, week) combinations
full_index = pd.MultiIndex.from_product([teams, weeks], names=['team', 'week'])

# Reindex the dataframe to include all rows
team_stats_25 = team_stats_25.set_index(['team', 'week']).reindex(full_index).reset_index()

In [ ]:
# Compute cumulative mean for previous games (excluding current game)
team_stats_25[[f"avg_{c}" for c in avg_cols]] = (
    team_stats_25.sort_values(by=['team', 'week'])
    .groupby("team")[avg_cols]
      .expanding()
      .mean()
      .shift(periods=1)
      .reset_index(level=0, drop=True)
)

## Retrieve Time of Possesion Stats

In [ ]:
for attempt in range(1, 5 + 1):
    try:
        # set up the browser
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        
        # visit T.O.P. Website
        driver.get("https://www.teamrankings.com/nfl/stat/average-time-of-possession-net-of-ot")

        table = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//*[@id='DataTables_Table_0']"))
                    )

        df = pd.read_html(table.get_attribute("outerHTML"))[0]

        driver.quit()
        break

    except Exception as e:
        driver.quit()
        print(f"Attempt {attempt} failed: {e}")
        if attempt == 5:
            print("No retries left — raising.")
            raise

C:\Users\tyler\AppData\Local\Temp\ipykernel_29608\26274015.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute("outerHTML"))[0]


In [ ]:
mapping = {'Buffalo': 'BUF',
 'LA Chargers': 'LAC',
 'Kansas City': 'KC',
 'Houston': 'HOU',
 'Detroit': 'DET',
 'Carolina': 'CAR',
 'Arizona': 'ARI',
 'New England': 'NE',
 'Jacksonville': 'JAX',
 'Chicago': 'CHI',
 'Tampa Bay': 'TB',
 'Atlanta': 'ATL',
 'NY Giants': 'NYG',
 'Denver': 'DEN',
 'Green Bay': 'GB',
 'San Francisco': 'SF',
 'Philadelphia': 'PHI',
 'Cleveland': 'CLE',
 'Dallas': 'DAL',
 'NY Jets': 'NYJ',
 'Indianapolis': 'IND',
 'Las Vegas': 'LV',
 'Minnesota': 'MIN',
 'Seattle': 'SEA',
 'Miami': 'MIA',
 'LA Rams': 'LA',
 'Baltimore': 'BAL',
 'New Orleans': 'NO',
 'Tennessee': 'TEN',
 'Washington': 'WAS',
 'Pittsburgh': 'PIT',
 'Cincinnati': 'CIN'}

df['team_abb'] = df['Team'].map(mapping)

In [ ]:
top_dict_24 = pd.Series(df['2024'].values, index=df['team_abb']).to_dict()
top_dict_25 = pd.Series(df['2025'].values, index=df['team_abb']).to_dict()

## Add stats for previous weeks in schedule

In [ ]:
desired_columns = ['season', 'week', 'team', 'opponent_team', 'avg_passing_epa', 'avg_passing_cpoe', 'avg_rushing_epa', 'avg_receiving_epa', 'avg_fg_pct', 'avg_pat_pct', 'avg_completions',
       'avg_attempts', 'avg_passing_yards', 'avg_passing_tds', 'avg_passing_interceptions', 'avg_sacks_suffered', 'avg_sack_yards_lost', 'avg_sack_fumbles_lost', 'avg_carries',
       'avg_rushing_yards', 'avg_rushing_tds', 'avg_rushing_fumbles_lost', 'avg_rushing_first_downs', 'avg_receiving_fumbles_lost', 'avg_def_tackles_solo', 'avg_def_tackles_with_assist',
       'avg_def_tackle_assists', 'avg_def_tackles_for_loss', 'avg_def_tackles_for_loss_yards', 'avg_def_fumbles_forced', 'avg_def_sacks', 'avg_def_sack_yards', 'avg_def_qb_hits',
       'avg_def_interceptions', 'avg_def_interception_yards', 'avg_def_pass_defended', 'avg_def_tds', 'avg_def_fumbles', 'avg_def_safeties', 'avg_penalties', 'avg_penalty_yards',
       'avg_punt_returns', 'avg_punt_return_yards', 'avg_kickoff_returns', 'avg_kickoff_return_yards', 'avg_fg_att', 'avg_points_scored', 'avg_points_against']

team_stats_24 = team_stats_24[desired_columns]
team_stats_25 = team_stats_25[desired_columns]


In [ ]:
team_stats = pd.concat([team_stats_24, team_stats_25])

In [ ]:
offensive = ['season','week', 'team', 'avg_passing_epa', 'avg_passing_cpoe', 'avg_rushing_epa', 'avg_receiving_epa', 'avg_fg_pct', 'avg_pat_pct', 'avg_completions',
       'avg_attempts', 'avg_passing_yards', 'avg_passing_tds', 'avg_passing_interceptions', 'avg_sacks_suffered', 'avg_sack_yards_lost', 'avg_sack_fumbles_lost', 'avg_carries',
       'avg_rushing_yards', 'avg_rushing_tds', 'avg_rushing_fumbles_lost', 'avg_rushing_first_downs', 'avg_receiving_fumbles_lost', 'avg_penalties', 'avg_penalty_yards',
       'avg_punt_returns', 'avg_punt_return_yards', 'avg_kickoff_returns', 'avg_kickoff_return_yards', 'avg_fg_att', 'avg_points_scored']

defensive = ['season', 'week', 'opponent_team', 'avg_def_tackles_solo', 'avg_def_tackles_with_assist', 'avg_def_tackle_assists', 'avg_def_tackles_for_loss', 'avg_def_tackles_for_loss_yards', 
        'avg_def_fumbles_forced', 'avg_def_sacks', 'avg_def_sack_yards', 'avg_def_qb_hits', 'avg_def_interceptions', 'avg_def_interception_yards', 'avg_def_pass_defended', 
        'avg_def_tds', 'avg_def_fumbles', 'avg_def_safeties', 'avg_points_against']

offensive_stats = team_stats[offensive]
defensive_stats = team_stats[defensive]

In [ ]:
offensive_stats

,season,week,team,avg_passing_epa,avg_passing_cpoe,avg_rushing_epa,avg_receiving_epa,avg_fg_pct,avg_pat_pct,avg_completions,...,avg_rushing_first_downs,avg_receiving_fumbles_lost,avg_penalties,avg_penalty_yards,avg_punt_returns,avg_punt_return_yards,avg_kickoff_returns,avg_kickoff_return_yards,avg_fg_att,avg_points_scored
0,2024.0,1,ARI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024.0,2,ARI,-1.184850,1.196628,4.984950,8.129718,1.00,1.0,21.000000,...,7.000000,0.00,5.000000,31.000000,1.000000,6.000000,3.000000,123.000000,2.000000,28.000000
2,2024.0,3,ARI,8.458119,9.371844,4.193653,14.001905,1.00,1.0,19.000000,...,11.000000,0.00,5.000000,41.500000,1.500000,18.000000,1.500000,61.500000,2.000000,34.500000
3,2024.0,4,ARI,3.761445,5.712925,0.461242,7.908119,1.00,1.0,19.666667,...,8.000000,0.00,4.666667,37.666667,1.000000,12.000000,1.000000,41.000000,2.000000,27.333333
4,2024.0,5,ARI,2.712770,4.387988,0.604399,6.289543,1.00,1.0,18.750000,...,9.250000,0.25,4.500000,48.750000,1.000000,10.000000,2.000000,63.250000,1.500000,24.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,2025.0,7,WAS,3.897319,3.270358,0.636833,9.342742,0.70,1.0,18.000000,...,7.666667,0.00,7.000000,51.833333,2.166667,31.000000,3.666667,120.833333,2.333333,26.333333
348,2025.0,8,WAS,0.489855,1.102632,1.373142,6.596160,0.70,1.0,17.714286,...,7.714286,0.00,7.571429,61.285714,2.000000,27.714286,4.285714,133.428571,2.000000,25.714286
349,2025.0,9,WAS,-0.496898,1.105433,0.950164,5.384997,0.70,1.0,18.125000,...,7.250000,0.00,7.000000,58.000000,1.875000,26.000000,4.125000,125.125000,1.750000,23.375000
350,2025.0,10,WAS,-0.589462,1.427237,0.856887,5.462606,0.70,1.0,17.888889,...,7.444444,0.00,7.111111,61.222222,1.888889,24.777778,4.000000,119.555556,1.555556,22.333333


In [ ]:
schedule_total['team'] = schedule_total['team'].str.strip().str.upper()
offensive_stats['team'] = offensive_stats['team'].str.strip().str.upper()
schedule_total['opponent'] = schedule_total['opponent'].str.strip().str.upper()
defensive_stats['opponent_team'] = defensive_stats['opponent_team'].str.strip().str.upper()
schedule_total = schedule_total[schedule['week'] > 1]

result = schedule_total.merge(offensive_stats, on=['season', 'week', 'team'], how='left')
result = result.merge(defensive_stats, left_on=['season', 'week', 'opponent'], right_on=['season', 'week','opponent_team'], how='left')

C:\Users\tyler\AppData\Local\Temp\ipykernel_29608\3419360845.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offensive_stats['team'] = offensive_stats['team'].str.strip().str.upper()
C:\Users\tyler\AppData\Local\Temp\ipykernel_29608\3419360845.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defensive_stats['opponent_team'] = defensive_stats['opponent_team'].str.strip().str.upper()
C:\Users\tyler\AppData\Local\Temp\ipykernel_29608\3419360845.py:5: UserWarning: Boolean Series key will be reindexed to 

### Final Touches and Download Data as CSV

In [ ]:
# Columns to fill
cols_to_fill = ['avg_fg_pct', 'avg_pat_pct']

# Fill NaN with 1 only for week 2 due to teams not having attempted a fg prior to the week
result.loc[result['week'] == 2, cols_to_fill] = result.loc[result['week'] == 2, cols_to_fill].fillna(1)


In [ ]:
# Split result into '24 and '25
result_24 = result[result['season'] == 2024]
result_25 = result[result['season'] == 2025]

In [ ]:
# Map time of possession stats
result_24['team_top'] = result_24['team'].map(top_dict_24)
result_24['opp_top'] = result_24['opponent_team'].map(top_dict_24)
result_25['team_top'] = result_25['team'].map(top_dict_25)
result_25['opp_top'] = result_25['opponent_team'].map(top_dict_25)

C:\Users\tyler\AppData\Local\Temp\ipykernel_29608\3603531570.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_24['team_top'] = result_24['team'].map(top_dict_24)
C:\Users\tyler\AppData\Local\Temp\ipykernel_29608\3603531570.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_24['opp_top'] = result_24['opponent_team'].map(top_dict_24)
C:\Users\tyler\AppData\Local\Temp\ipykernel_29608\3603531570.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [ ]:
# Recombine the results years
result = pd.concat([result_24, result_25])

In [ ]:
result

,game_id,season,week,team,opponent,points,total_line,avg_passing_epa,avg_passing_cpoe,avg_rushing_epa,...,avg_def_qb_hits,avg_def_interceptions,avg_def_interception_yards,avg_def_pass_defended,avg_def_tds,avg_def_fumbles,avg_def_safeties,avg_points_against,team_top,opp_top
0,2024_02_BUF_MIA,2024,2,BUF,MIA,31.0,48.5,9.161098,9.891942,3.497710,...,5.0,0.0,0.0,3.0,0.0,0.0,0.0,34.0,30:34,31:36
1,2024_02_LV_BAL,2024,2,LV,BAL,26.0,41.0,-1.242705,1.044023,-18.512759,...,5.0,0.0,0.0,2.0,0.0,0.0,0.0,10.0,29:20,31:12
2,2024_02_LAC_CAR,2024,2,LAC,CAR,26.0,39.0,-4.875880,-0.624557,0.271463,...,6.0,1.0,3.0,5.0,0.0,0.0,0.0,22.0,30:23,27:13
3,2024_02_NO_DAL,2024,2,NO,DAL,44.0,47.0,7.993491,10.192335,4.767348,...,5.0,2.0,2.0,6.0,0.0,1.0,0.0,47.0,28:08,29:16
4,2024_02_TB_DET,2024,2,TB,DET,20.0,51.5,20.591765,12.676064,-6.825627,...,2.0,0.0,0.0,2.0,0.0,0.0,0.0,37.0,31:41,31:46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,2025_18_DAL_NYG,2025,18,NYG,DAL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30:43,NaN
1020,2025_18_WAS_PHI,2025,18,PHI,WAS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30:47,NaN
1021,2025_18_BAL_PIT,2025,18,PIT,BAL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27:19,NaN
1022,2025_18_SEA_SF,2025,18,SF,SEA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29:45,NaN


In [ ]:
import os

try:
    # Works when running as a .py script
    base_dir = os.path.dirname(__file__)
except NameError:
    # Fallback for Jupyter, Python in Excel, or interactive environments
    base_dir = os.getcwd()

data_path = os.path.join(base_dir, '..', 'Data', 'NFLData.csv')

os.makedirs(os.path.dirname(data_path), exist_ok=True)
result.to_csv(data_path, index=False)